In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


def load_model():
    model_id = "microsoft/phi-4-reasoning"
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            trust_remote_code=True,
            dtype=torch.bfloat16,
            device_map="auto",
        )
        return tokenizer, model
    except Exception as e:
        raise RuntimeError(f"Failed to load any Phi-4 model. Last error: {e}")


def solve(problem: str):
    tokenizer, model = load_model()
    messages = [
        {"role": "system", "content": "You are Phi, a language model trained by Microsoft to help users. Your role as an assistant involves thoroughly exploring questions through a systematic thinking process before providing the final precise and accurate solutions. This requires engaging in a comprehensive cycle of analysis, summarizing, exploration, reassessment, reflection, backtracing, and iteration to develop well-considered thinking process. Please structure your response into two main sections: Thought and Solution using the specified format: <think> {Thought section} </think> {Solution section}. In the Thought section, detail your reasoning process in steps. Each step should include detailed considerations such as analysing questions, summarizing relevant findings, brainstorming new ideas, verifying the accuracy of the current steps, refining any errors, and revisiting previous steps. In the Solution section, based on various attempts, explorations, and reflections from the Thought section, systematically present the final solution that you deem correct. The Solution section should be logical, accurate, and concise and detail necessary steps needed to reach the conclusion. Now, try to solve the following question through the above guidelines:"},
        {"role": "user", "content": problem},
    ]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")

    gen_ids = model.generate(
        inputs.to(model.device),
        max_new_tokens=2048,
        temperature=0.8,
        top_p=0.95,
        top_k=50,
        do_sample=True,
    )
    output_text = tokenizer.decode(
        gen_ids[0], skip_special_tokens=True
    )
    print(output_text)

In [ ]:
problem = (
        "A bat and a ball cost $1.10 in total, and the bat costs $1.00 more than the ball. "
        "How much does the ball cost?"
    )
solve(problem)